- in the previous lesson you gained a conceptual understanding of the system architecture for Carla, our autonomous vehicle
- next you're going to see how this software is implemented and even practice implementing some of the software yourself
- Carla's software uses the Robot Operating System (ROS) which is an Open Source Robotics framework that provides libraries and tools for working with hardware and for passing messages between components
- ROS provides scaffolding to integrate the components of the car

- building a robot used to be a long and cumbersome process--all the components had to essentially be built from scratch
- in this lesson, we'll be giving you an introduction to robot operating system or ROS, which is a software framework that greatly simplifies robot development
- there are many advantages to developing robots with ROS, but before we dive deep, I think we should explain what ROS actually is


- ROS is an open source software framework for robotics development
- to help you get a better understanding of what ROS is, let's illustrate some of its components and features
- ROS is not an operating system in the typical sense, but like an OS, it provides a means of talking to hardware without writing your own device drivers
- it also provides a way for different processes to communicate with one another via message passing
- lastly, ROS features a sleek build and package management system allowing you to develop and deploy software with ease
- ROS also has tools for visualization, simulation and analysis, as well as extensive community support and an interface to numerous powerful software libraries


- check out this awesome [short documentary on ROS](https://www.bloomberg.com/news/videos/2017-05-17/the-silicon-valley-startup-creating-robot-dna-video)

# Brief History of ROS

- development of ROS began in the mid 2000s as Project Switchyard in Stanford's Artificial Intelligence Laboratory
- in 2007, ROS became a formal entity with support from Willow Garage
- since 2013, the Open Source Robotics Foundation has been maintaining and developing ROS


- one of the primary motivations for developing ROS was the recognition that researchers were constantly reinventing the wheel
- there weren't many reusable robotics software components that could be used as a starting point for a project
- in addition, because different groups were all working on custom solutions, it was difficult for them to share code and ideas
- it was also difficult to compare results
- ROS aims to facilitate the development process by eliminating these problems


- there are drones, kinematic arms, wheeled robots, and even bipedal robots out there using ROS every day

# Nodes and Topics

- pretty much all the robots we're interested in for this course share the same basic characteristics
- they contain sensors for perceiving the world around them, software for making high level decisions, and motors and controllers for actuation
- ROS provides a powerful communications system allowing these different components to communicate with one another
- here, we'll introduce this communication system by considering a generic robot
  - this robot could be a drone, a mobile base or even a robotic arm
  - while these robots may be composed of very different physical components, they are all built to perform these same three high level steps of perception, decision making and actuation


- on the software side, ROS manages these three complex steps by breaking each of them down into many small Unix processes called **nodes**
- typically, each node on the system is responsible for one small and relatively specific portion of the robot's overall functionality
  - for example, there may be nodes for each sensor and actuator in the system, as well as nodes for things like position estimation, behavior execution and motor control


- at the center of this collection of nodes is the **ROS Master** which acts as a sort of manager of all the nodes
- the ROS Master maintains a registry of all the active nodes on a system
- it then allows each node to discover other nodes in the system and establish lines of communication with them


- in addition to allowing nodes to locate one another and communicate, the ROS Master also hosts what's called the parameter server
- the parameter server is typically used to store parameters and configuration values that are shared amongst the running nodes
  - for example, a mobile robot's wheel radius may be used by one node to estimate position and by another to calculate speed
  - rather than storing the same information in multiple places, nodes can look up values as needed


- nodes can also share data with one another by passing messages over what are called **topics**
- a topic is simply a named bus which you can think of as a pipe between nodes through which messages flow
- in order to send a message on a topic, we say that a node must *publish* to it
- likewise, to receive a message on a topic, a node must *subscribe* to it
- it's important to note that each node may simultaneously publish and subscribe to a wide variety of topics
- taken together, this network of nodes connected by topics is called publish subscribe or a pub sub architecture

# Message Passing

- each ROS distribution comes with a wide variety of predefined message types which are available for your use
- there are message types for communicating physical quantities, such as positions, velocities, accelerations, rotations, and durations
- there are also messages for communicating sensor readings, such as laser scans, images, point clouds, inertial measurements, and more
- the number of message types available in a full ROS install is actually quite staggering
  - doing a simple search in my system, I was able to count over 200 different message types
- however, even with so many useful message types available, there will still be times when you want to define your own--we'll tackle that later on in this lesson


- it's important to note that although the named messages would seem to imply text based contents, they can in fact contain any kind of data
- here's an example of the message types associated with the topics we looked at earlier
  - in this case, the topic called camera_images is transporting image messages from the camera node to the position estimator node
  - the wheel encoder's topic is transporting rotation messages from the wheel encoder node to the position estimator node and so on

# Services

- passing messages over topics between publishers and subscribers is useful but it's not a one size fits all communications solution
- there are times when a request-response pattern is useful
  - for these types of interactions, ROS provides what are called **services**
- like topics, services allow the passing of messages between nodes
- however, unlike topics, a service is not a bus and there are no publishers or subscribers associated with them
- instead, nodes interacting via services do so on a one to one basis using a request and response messages


- let's take a look at how a service might be used in the example we've been developing
- let's say that the behavior executor node wants to capture an image from the camera
- we could just have it subscribe to the camera images topic
- it would then receive a new image each time the camera node publishes one
- however, this might be undesirable if the executor only wants to capture an image every once in a while
- instead of subscribing to the camera images topic here, we'll define a new service for the camera node--let's call it capture_image
- the request message definition might be a custom message type called exposure_time and the response might just be an image message
- by issuing a service call to the capture image service, the behavior executor node or any other node for that matter, is able to request the camera should capture an image with a given exposure time setting
- the camera node will then respond to the request by sending a message containing the resulting image

# Compute Graph

- these diagrams of nodes and topics and how they're all connected and are frequently referred to as compute graphs
- visualizing the compute graph is very useful for understanding what nodes exist, and how they communicate with one another
- ROS provides a tool called `rqt_graph` for showing the compute graph of a system
- a moderately complex robot will likely have dozens of nodes, even more topics and quite a few services

# Turtlesim Overview

- now that we have an understanding of some of the fundamental concepts of ROS, we can drive right into the command line where we will create cute little turtle robots with a package called Turtlesim
- cuteness aside the use of turtles and robotics goes way back to the 1940s when early roboticist William Gray Walter created some of the first autonomous devices, turtle robots which he called Elmer and Elsie
- and in the 1960s at MIT, Seymour Papert used turtle robots in robotics education
  - Seymour's robots could perform a few basic functions--they could move forward or backwards by a given distance, rotate by a given angle, or they could drop the retractable pen on their bellies allowing them to draw as they moved
  - these primitive functions could be triggered simultaneously allowing the robots to move along complex trajectories and make sophisticated drawings
  - he is also known as the creator and evangelist for the educational programming language LOGO


- this tradition of turtles in robotics is alive and well today
- in fact, each recent version of ROS has been named after some sort of turtle
- in addition to this turtle-centric naming convention, the OSRF also adds a new turtle to `turtle_sim` with each release

# Your Virtual Machine

- to follow along with the exercises in these lessons, you will need to be running Ubuntu Linux with ROS installed
- so that everyone can follow along in the same environment we are providing you with a virtual machine (VM) that is all setup with everything you need
- so, find yourself a good internet connection and download the [compressed VM disk image](https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/Udacity_VM_Base_V1.0.0.zip)


- in the zipped archive in addition to the VM disk image, there are two other small configuration files that you may or may not need
- we recommend using [VirtualBox](https://www.virtualbox.org/wiki/Downloads) to run the VM
- this will enable you to easily use port forwarding, so you can run the final project simulator in your native operating system (the host) while using the VM for running ROS code
- import your VM image to VirtualBox
  - download and install VirtualBox
  - download the image from supplied link
  - unzip the image
  - open VirtualBox Application
  - click File > Import Appliance
  - click the folder icon on the right and navigate to your unzipped image (the .ovf file)
  - follow the prompts to import the image
  - from the VirtualBox Manager, select your VM and press start


- before getting your VM up and running, you will need to make sure that you allocate enough resources to allow it to operate efficiently
- with your VM shut down, navigate to the VM settings by clicking on the icon in the VirtualBox Manager
- in the "System" cateogory Look for tabs labeled "Motherboard" and "Processor"; this is where you will change the amount of RAM and number of cores that you allocate from your own machine to the VM
- if you have the resources on your machine, we recommend allocating at least 2 processor cores and 4 GB of RAM
- if you are able to allocate more resources, feel free to do so
- the more you allocate the better performance you will get!
- you're now all set to boot up your VM!


- upon your first boot you will be prompted to choose a keyboard layout of your choice
- once you select a keyboard you will not be prompted for this again
- if you would like to change your option you can reset this feature by entering `udacity_reset` in a terminal of your choice and restarting the VM


- once you are up and running, you might be asked to input a password to enter the VM--**the password for the VM is udacity-nd**
- to open a terminal in your VM press `Ctrl-Alt-t` (or `ctrl-option-t` on a Mac


- performing a native install of Linux and ROS (unsupported)
  - some of you may choose not to use the virtual machine for a variety of reasons
  - if you choose to install ROS (Robot Operating System) on your own, it is recommended that you use [Ubuntu 16.04 LTS](http://releases.ubuntu.com/16.04/) as your operating system
  - to install ROS, please visit: [ROS documentation](http://wiki.ros.org/)
  - this method is not supported by Udacity

# Source the ROS Environment

- in order to launch the `turtlesim` package, you must first ensure that all of the ROS environment variables have been correctly set
- each ROS distribution provides a handy little script called `setup.bash` specifically for this purpose
- the script is hidden away in the opt ROS directory and you can run it with the bash command `source /opt/ros/kinetic/setup.bash`
- among other things, the environment variables tell our bash shell where ROS commands and packages can be found
- one simple way to determine if the environment variables have been set is to use the bash tab completion feature--typing ROS followed by double tapping the tab key gives us an extensive list of all commands beginning with the characters R-O-S
  - this list would be much shorter had we not sourced as `setup.bash`
- having to source the ROS `setup.bash` script for each new terminal you open is quickly going to become painful
  - to combat the monotony we can add the command to source `setup.bash` to our `.bashrc`


- make sure you use the bash command `source` rather than `./`
- there’s a subtle distinction between the two commands, in that `source` executes the script in the current session, while `./` will start a new session, containing a copy of the current environment
- when a script executed via `./` is exited, all environment variables set by it will be lost--we don’t want this
- for more information on environment variables and terminal sessions, please see [here](https://help.ubuntu.com/community/EnvironmentVariables)


- while we won’t be discussing all of the environment variables that have been added or changed, there are a few worth mentioning
  - `ROS_ROOT=/opt/ros/kinetic/share/ros` the path where core ros packages are stored
  - `PATH=/opt/ros/kinetic/bin:...` the path to the ROS binaries, which we will be using throughout this lesson
  - `ROS_DISTRO=kinetic` which distribution of ROS is being used--in this case, we are using kinetic
  - `PYTHONPATH=/opt/ros/kinetic/lib/python2.7/dist-packages` the path to the ROS python packages, which we will be using next lesson
- more information about the environment variables mentioned above, the ones not mentioned here, and others can be found [here](http://wiki.ros.org/ROS/EnvironmentVariables)

# Run Turtlesim

- now that you’ve added ROS’ environment variables to your terminal session, you can run the turtlesim package


- before any ROS nodes can be run, the Master process must be started
- the Master process is responsible for the following (and more):
  - providing naming and registration services to other running nodes
  - tracking all publishers and subscribers
  - aggregating log messages generated by the nodes
  - facilitating connections between nodes
- to run the master process, you must execute the command `roscore`
- to terminate the ROS Master process, simply send the SIGINT signal to the process by typing Ctrl-C in the roscore terminal window


- now that the ROS master is running, we can run our first two ROS nodes
- to do so, we will execute the `rosrun` command in a new terminal window, passing as parameters the name of the package we wish to run, and the name of the actual node
  - first we will start the `turtlesim_node`, in the `turtlesim` package using the following command:
```shell
rosrun turtlesim turtlesim_node
```
  - next, we will start the `turtle_teleop_key node`, also from the `turtlesim` package
```
 rosrun turtlesim turtle_teleop_key
```

- by using the arrow keys with the `turtle_teleop_key` node’s console selected, we are able to move the turtle in turtlesim

# Turtlesim Comms: List Nodes

- to get a list of all nodes that are active and have been registered with the ROS Master, we can use the command `rosnode list`


- we can see that there are three active nodes that have been registered with the ROS Master:
  - `/rosout` node is launched by roscore and it's subscribes to the standard /rosout topic, the topic to which all nodes send log messages
  - `/teleop_turtle` is our keyboard teleop node
    - notice that it’s not named `turtle_teleop_key`
    - there’s no requirement that a node’s broadcasted name is the same as the name of it’s associated executable
  - `/turtlesim` is the node name associated with the `turtlebot_sim` node

# Turtlesim Comms: List Topics

- in a similar fashion, we are able to query the ROS Master for a list of all topics
- to do so, we use the command `rostopic list`


- `/rosout_agg` is aggregated feed of messages published to /rosout
- `/turtle1/cmd_vel` is topic upon which velocity commands are sent/received
  - publishing a velocity message to this topic will command turtle1 to move
- `/turtle1/color_sensor` each turtle in turtlesim is equipped with a color sensor, and readings from the sensor are published to this topic
- `/turtle1/pose` the position and orientation of turtle1 are published to this topic

# Turtlesim Comms: Get Topic Info

- if we wish to get information about a specific topic, who is publishing to it, subscribed to it, or the type of message associated with it, we can use the command `rostopic info`


- as would be expected, there are two nodes registered on this topic
- our publisher, the `teleop_turtle` node, and our subscriber, the `turtlesim node`
- additionally, we can see that the type of message used on this topic is `geometry_msgs/Twist`

# Turtlesim Comms: Message Information

- the `rosmsg info` command prints out message type information to the console


- let’s get some more information about the `geometry_msgs/Twist` message on the `/turtle1/cmd_vel` topic
  - we can see that a `Twist` message consists nothing more than two `Vector3` messages
    - one for linear velocity, and another for angular velocities, with each velocity component being represented by a float64


- sometimes, the message definition won’t provide an ample amount of detail about a message type
- for example, in the example above, how can we be sure that linear and angular vectors above refer to velocities, and not positions?
- one way to get more detail would be to look at the comments in the message’s definition file


- to do so, we can issue the following command: `rosed geometry_msgs Twist.msg`
- `rosed` is a simple bash command that allows you to easily view and edit files in your ros environment
- more information about `rosed`, including how to select which editor is used by default can be found [here](http://wiki.ros.org/ROS/Tutorials/UsingRosEd)

# Turtlesim Comms: Echo a Topic

- sometimes it may be useful to look at a topic’s published messages in real time
- to do so, we can use the command `rostopic echo`


- let’s take a look at the `/turtle1/cmd_vel topic`

```shell
rostopic echo /turtle1/cmd_vel
```
- if we then command the turtle to move from the `turtle_teleop_key` window, we will be able to see the output message in real-time

# Recap

- in this lesson, you've seen a high level overview of the concepts behind ROS
- you know now how the ROS master process functions as a parameter server, and how it enables nodes to be connected over topics
- also, you've seen how ROS nodes communicate over these topics using message passing and services
- this Pub/Sub architecture is a powerful and flexible design paradigm that will allow you to build a variety of robotics applications
- additionally, you've explored the turtlesim package using ROS' powerful command line tools
  - these tools will be helpful to you as you build and debug ROS-based systems going forward